<!-- Banner Image -->
<img src="https://uohmivykqgnnbiouffke.supabase.co/storage/v1/object/public/landingpage/brevdevnotebooks.png" width="100%">

<!-- Links -->
<center>
  <a href="https://console.brev.dev" style="color: #06b6d4;">Console</a> •
  <a href="https://brev.dev" style="color: #06b6d4;">Docs</a> •
  <a href="/" style="color: #06b6d4;">Templates</a> •
  <a href="https://discord.gg/NVDyv7TUgJ" style="color: #06b6d4;">Discord</a>
</center>

# ⚡ Fine-Tuning Performance Comparison: Optimization Techniques

## Real Benchmarks. Same Model. Same Dataset. YOUR GPU.

<div style="background: linear-gradient(135deg, #f093fb 0%, #f5576c 100%); padding: 30px; border-radius: 10px; color: white; margin-bottom: 20px;">
  <h2 style="margin-top: 0; color: white;">🎯 What You'll Discover</h2>
  <p style="font-size: 18px; line-height: 1.6;">
    <strong>Stop guessing. Start measuring.</strong><br/><br/>
    We'll train <strong>the same model</strong> with <strong>the same dataset</strong> using two approaches:<br/>
    ⚡ <strong>Unsloth</strong> - Optimized kernels built on HuggingFace<br/>
    🤗 <strong>Standard HuggingFace PEFT</strong> - Default configuration<br/>
    <br/>
    <strong>🔥 GPU starts training in 60 seconds. Side-by-side results in 10 minutes.</strong>
  </p>
</div>

## 💡 About This Comparison

Both approaches use **HuggingFace's transformers and PEFT libraries** - the foundation of modern LLM fine-tuning.

**What we're measuring:**
- **Standard Configuration**: Out-of-the-box HuggingFace PEFT + Trainer
- **Optimized Configuration**: Unsloth's custom kernels (Flash Attention, optimized checkpointing)

This helps you understand the performance impact of optimization layers and choose the right approach for your use case.

## 📋 Prerequisites

- **GPU**: NVIDIA GPU with 16GB+ VRAM (A100, H100, L40S, RTX 4090, RTX 3090)
- **CUDA**: 11.8+ or 12.1+
- **Python**: 3.10+
- **Disk Space**: 20GB free (for models + datasets)

---

## 🎬 Quick Start: What Happens Next

The next few cells will:
1. **Install Unsloth** (~30 sec)
2. **Load Qwen 1.5B model** (~15 sec)
3. **Start training** immediately (2-3 min)
4. **Train with vanilla HF** (same config)
5. **Compare ALL metrics** side-by-side

**Total time: ~10 minutes for complete comparison** ⚡

---

#### 💬 Questions? Join us on [Discord](https://discord.gg/NVDyv7TUgJ) or reach out on [X/Twitter](https://x.com/brevdev)

**📝 Notebook Tips**: Press `Shift + Enter` to run cells. A `*` means running, a number means complete.

---

## 1. Verify GPU Setup 🎮

Let's make sure your NVIDIA GPU is ready for fine-tuning!


In [ ]:
# Cell 1: GPU Verification
# =========================
# Quick check that GPU is available and has enough memory

import subprocess
import sys
import os

print("="*80)
print("🎮 GPU STATUS CHECK")
print("="*80 + "\n")

# Check nvidia-smi
try:
    result = subprocess.run(
        ["nvidia-smi", "--query-gpu=name,memory.total,driver_version", "--format=csv,noheader"],
        capture_output=True,
        text=True,
        timeout=5
    )
    
    if result.returncode == 0:
        gpu_info = result.stdout.strip().split(", ")
        print(f"✅ GPU Detected: {gpu_info[0]}")
        print(f"✅ VRAM: {gpu_info[1]}")
        print(f"✅ Driver: {gpu_info[2]}")
        
        # Check if enough memory
        vram_str = gpu_info[1].replace(' MiB', '').strip()
        try:
            vram_gb = float(vram_str) / 1024
            if vram_gb < 16:
                print(f"\n⚠️  Warning: Your GPU has {vram_gb:.1f}GB VRAM.")
                print("   This notebook recommends 16GB+ for full comparison.")
                print("   Training may still work with smaller models or reduced batch sizes.")
        except:
            print("   (Could not parse VRAM size)")
    else:
        print("❌ nvidia-smi failed. Is NVIDIA driver installed?")
        sys.exit(1)
        
except FileNotFoundError:
    print("❌ nvidia-smi not found. Please install NVIDIA drivers.")
    sys.exit(1)
except Exception as e:
    print(f"❌ Error checking GPU: {e}")
    sys.exit(1)

# Check PyTorch CUDA
print("\n" + "-"*80)
print("Checking PyTorch CUDA support...\n")

try:
    import torch
    if torch.cuda.is_available():
        print(f"✅ PyTorch {torch.__version__}")
        print(f"✅ CUDA {torch.version.cuda}")
        print(f"✅ {torch.cuda.device_count()} GPU(s) available")
    else:
        print("❌ PyTorch installed but CUDA not available")
        print("💡 Try: pip install torch --index-url https://download.pytorch.org/whl/cu121")
        sys.exit(1)
except ImportError:
    print("⚠️  PyTorch not found. Installing with CUDA support...")
    print("   This may take 1-2 minutes...\n")
    try:
        # Install PyTorch with CUDA support
        result = subprocess.run(
            [sys.executable, "-m", "pip", "install", "torch", "torchvision", "torchaudio", 
             "--index-url", "https://download.pytorch.org/whl/cu121"],
            capture_output=True,
            text=True,
            timeout=300
        )
        if result.returncode != 0:
            print(f"⚠️  Installation had issues: {result.stderr[:200]}")
            print("   Trying alternative installation method...")
            subprocess.run([sys.executable, "-m", "pip", "install", "torch"], check=False)
        
        import torch
        print(f"✅ PyTorch {torch.__version__} installed")
        if torch.cuda.is_available():
            print(f"✅ CUDA {torch.version.cuda} available")
        else:
            print("⚠️  PyTorch installed but CUDA not detected")
    except Exception as e:
        print(f"⚠️  PyTorch installation failed: {e}")
        print("💡 Please install manually: pip install torch --index-url https://download.pytorch.org/whl/cu121")
        sys.exit(1)

print("\n" + "="*80)
print("✅ GPU READY FOR TRAINING!")
print("="*80 + "\n")


## 2. Install Dependencies ⚡

We'll install Unsloth and its dependencies. Unsloth is built on top of HuggingFace's transformers library and adds optimized CUDA kernels for faster training.

**What Unsloth adds:**
- Flash Attention 2 kernels
- Optimized gradient checkpointing  
- Memory-efficient operations

We'll train with both configurations to measure the real-world impact.


In [ ]:
# Cell 2: Install Unsloth & Dependencies
# =======================================

import time

print("="*80)
print("⚡ INSTALLING UNSLOTH & DEPENDENCIES")
print("="*80 + "\n")

install_start = time.time()

packages = [
    "unsloth",
    "transformers",
    "datasets",
    "peft",
    "trl",
    "accelerate",
    "bitsandbytes",
    "matplotlib",
    "seaborn",
    "pandas"
]

print("📦 Installing packages (this may take 1-2 minutes)...\n")

failed_packages = []
for package in packages:
    print(f"   Installing {package}... ", end="", flush=True)
    try:
        result = subprocess.run(
            [sys.executable, "-m", "pip", "install", package, "-q"],
            capture_output=True,
            timeout=180,
            check=False
        )
        if result.returncode == 0:
            print("✅")
        else:
            print("⚠️ ")
            failed_packages.append(package)
    except Exception as e:
        print(f"⚠️ ")
        failed_packages.append(package)

if failed_packages:
    print(f"\n⚠️  Some packages had issues: {', '.join(failed_packages)}")
    print("   The notebook may still work. If you see errors, install manually.")

install_time = time.time() - install_start

print(f"\n✅ Installation complete in {install_time:.1f}s")
print("="*80 + "\n")


## 3. Load Model & Dataset 📦

**Model:** Qwen2.5-1.5B-Instruct (fast to train, production-quality)  
**Dataset:** OpenHermes-2.5 (5K samples, high-quality instruction data)


In [ ]:
# Cell 3: Load Model + Dataset
# =============================

from unsloth import FastLanguageModel
from datasets import load_dataset

print("="*80)
print("📦 LOADING MODEL & DATASET")
print("="*80 + "\n")

MODEL_NAME = "unsloth/Qwen2.5-1.5B-Instruct"
MAX_SEQ_LENGTH = 2048

print(f"[1/2] Loading model: {MODEL_NAME}...\n")
model_start = time.time()

try:
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=MODEL_NAME,
        max_seq_length=MAX_SEQ_LENGTH,
        dtype=None,
        load_in_4bit=True,
    )
    
    model_time = time.time() - model_start
    print(f"   ✅ Model loaded in {model_time:.1f}s")
    
    if torch.cuda.is_available():
        memory_allocated = torch.cuda.memory_allocated(0) / 1e9
        print(f"   📊 GPU Memory: {memory_allocated:.2f} GB\n")
        
except Exception as e:
    print(f"   ❌ Model loading failed: {e}")
    raise

print(f"[2/2] Loading dataset: OpenHermes-2.5 (5,000 samples)...\n")
dataset_start = time.time()

try:
    dataset = load_dataset("teknium/OpenHermes-2.5", split="train[:5000]")
    dataset_time = time.time() - dataset_start
    print(f"   ✅ Dataset loaded in {dataset_time:.1f}s")
    print(f"   📝 {len(dataset)} training samples\n")
except Exception as e:
    print(f"   ⚠️  Primary dataset failed, using backup...")
    dataset = load_dataset("mlabonne/guanaco-llama2-1k", split="train")
    print(f"   ✅ Backup dataset: {len(dataset)} samples\n")

print("="*80)
print("✅ READY TO START TRAINING!")
print("="*80 + "\n")


## 4. 🔥 Training Run 1: Optimized Configuration (Unsloth)

### GPU ACTIVE NOW!

**Configuration** (identical for both runs):
- LoRA rank: 16, alpha: 32
- Batch size: 2 × 4 = 8 effective
- Learning rate: 2e-4, Steps: 60
- 4-bit quantization (QLoRA)

This run uses Unsloth's optimized kernels on top of HuggingFace transformers.


In [ ]:
# Cell 4: Train with Unsloth
# ===========================

from unsloth import is_bfloat16_supported
from trl import SFTTrainer
from transformers import TrainingArguments

print("="*80)
print("⚡ RUN 1: OPTIMIZED CONFIGURATION")
print("="*80)
print("\n🔥 Training with Unsloth optimization layer...\n")

# Configure LoRA
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
)

print("✅ LoRA configured\n")

# Format dataset
def format_prompts(examples):
    texts = []
    for convs in examples["conversations"]:
        try:
            text = tokenizer.apply_chat_template(convs, tokenize=False, add_generation_prompt=False)
            texts.append(text)
        except:
            texts.append(str(convs))
    return {"text": texts}

dataset = dataset.map(format_prompts, batched=True)
print("✅ Dataset formatted\n")

# Reset GPU stats
torch.cuda.reset_peak_memory_stats()
training_start = time.time()

# Training!
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=MAX_SEQ_LENGTH,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=60,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs_unsloth",
        report_to="none",
    ),
)

trainer_stats = trainer.train()

# Collect metrics
unsloth_time = time.time() - training_start
unsloth_memory = torch.cuda.max_memory_allocated(0) / 1e9
unsloth_loss = trainer_stats.training_loss

print("\n" + "="*80)
print("✅ OPTIMIZED TRAINING COMPLETE!")
print("="*80)
print(f"\n⏱️  Time: {unsloth_time/60:.2f} min ({unsloth_time:.1f}s)")
print(f"💾 Peak Memory: {unsloth_memory:.2f} GB")
print(f"📉 Final Loss: {unsloth_loss:.4f}")

# Save checkpoint
model.save_pretrained("unsloth_lora_model")
tokenizer.save_pretrained("unsloth_lora_model")
print(f"💾 Checkpoint saved\n")

# Store results
unsloth_results = {
    "method": "Unsloth",
    "time_seconds": unsloth_time,
    "memory_gb": unsloth_memory,
    "loss": unsloth_loss,
}

print("="*80 + "\n")


## 5. Training Run 2: Standard Configuration (HuggingFace PEFT) 🤗

Now train the **same model** using standard HuggingFace PEFT + Trainer.

**Why compare?**
- Standard config offers maximum flexibility and compatibility
- Works with any model architecture supported by transformers
- Easier to customize for research and experimentation
- Industry-standard approach used in production worldwide

Let's measure the performance characteristics of both approaches.


In [ ]:
# Cell 5: Train with HuggingFace (Baseline)
# ===========================================

from transformers import (
    AutoModelForCausalLM, AutoTokenizer, Trainer,
    BitsAndBytesConfig, DataCollatorForLanguageModeling
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

print("="*80)
print("🤗 RUN 2: STANDARD CONFIGURATION")
print("="*80 + "\n")

# Clear GPU
del model, trainer
torch.cuda.empty_cache()

print("[1/3] Loading base model...")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16 if is_bfloat16_supported() else torch.float16,
    bnb_4bit_use_double_quant=True,
)

hf_model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2.5-1.5B-Instruct",
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)
hf_tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-1.5B-Instruct", trust_remote_code=True)
hf_tokenizer.pad_token = hf_tokenizer.eos_token
hf_model = prepare_model_for_kbit_training(hf_model)
print("   ✅ Model loaded\n")

print("[2/3] Configuring LoRA (identical to optimized run)...")
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)
hf_model = get_peft_model(hf_model, lora_config)
print("   ✅ LoRA configured\n")

print("[3/3] Preparing dataset...")
hf_dataset = load_dataset("teknium/OpenHermes-2.5", split="train[:5000]")
hf_dataset = hf_dataset.map(format_prompts, batched=True)

def tokenize_fn(examples):
    return hf_tokenizer(examples["text"], truncation=True, max_length=2048, padding="max_length")

tokenized = hf_dataset.map(tokenize_fn, batched=True, remove_columns=hf_dataset.column_names)
print("   ✅ Dataset ready\n")

print("🔥 Training with standard HuggingFace PEFT configuration...\n")

hf_training_args = TrainingArguments(
    output_dir="outputs_huggingface",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    max_steps=60,
    learning_rate=2e-4,
    fp16=not is_bfloat16_supported(),
    bf16=is_bfloat16_supported(),
    logging_steps=10,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    warmup_steps=5,
    save_steps=0,
    report_to="none",
)

data_collator = DataCollatorForLanguageModeling(tokenizer=hf_tokenizer, mlm=False)
hf_trainer = Trainer(
    model=hf_model,
    args=hf_training_args,
    train_dataset=tokenized,
    data_collator=data_collator,
)

torch.cuda.reset_peak_memory_stats()
hf_start = time.time()

hf_trainer_stats = hf_trainer.train()

hf_time = time.time() - hf_start
hf_memory = torch.cuda.max_memory_allocated(0) / 1e9
hf_loss = hf_trainer_stats.training_loss

print("\n" + "="*80)
print("✅ STANDARD TRAINING COMPLETE!")
print("="*80)
print(f"\n⏱️  Time: {hf_time/60:.2f} min ({hf_time:.1f}s)")
print(f"💾 Peak Memory: {hf_memory:.2f} GB")
print(f"📉 Final Loss: {hf_loss:.4f}")

hf_model.save_pretrained("huggingface_lora_model")
print(f"💾 Checkpoint saved\n")

hf_results = {
    "method": "HuggingFace",
    "time_seconds": hf_time,
    "memory_gb": hf_memory,
    "loss": hf_loss,
}

print("="*80)
print("🎉 BOTH TRAINING RUNS COMPLETE!")
print("   Ready to compare performance characteristics...")
print("="*80 + "\n")


## 6. 📊 Performance Analysis

### Understanding the Trade-offs

We trained the **same model** with identical hyperparameters using two configurations:
- **Optimized**: Unsloth's custom kernels
- **Standard**: HuggingFace PEFT default

Only variable: the optimization layer.

Let's analyze the performance characteristics...


In [ ]:
# Cell 6: Comparison & Visualization
# ====================================

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

print("="*80)
print("📊 COMPARISON DASHBOARD")
print("="*80 + "\n")

# Create dataframe
comparison_df = pd.DataFrame([unsloth_results, hf_results])
comparison_df['speedup'] = hf_results['time_seconds'] / comparison_df['time_seconds']
comparison_df['memory_savings_pct'] = (hf_results['memory_gb'] - comparison_df['memory_gb']) / hf_results['memory_gb'] * 100

# Display table
print("📋 RAW METRICS:\n")
print(f"{'Method':<15} {'Time (s)':<12} {'Memory (GB)':<15} {'Loss':<10} {'Speedup'}")
print("-"*70)
for _, row in comparison_df.iterrows():
    print(f"{row['method']:<15} {row['time_seconds']:<12.1f} {row['memory_gb']:<15.2f} {row['loss']:<10.4f} {row['speedup']:.2f}×")

# Key findings
speedup = unsloth_results['speedup']
mem_saved = comparison_df.loc[comparison_df['method']=='Unsloth', 'memory_savings_pct'].values[0]
time_saved = hf_results['time_seconds'] - unsloth_results['time_seconds']

print("\n" + "="*80)
print("💡 PERFORMANCE CHARACTERISTICS")
print("="*80)
print(f"\n⚡ Training Speed:")
print(f"   • Optimized: {unsloth_results['time_seconds']:.1f}s")
print(f"   • Standard: {hf_results['time_seconds']:.1f}s")
print(f"   • Speedup: {speedup:.2f}× with optimized kernels")
print(f"   • Time saved: {time_saved:.1f}s per run")
print(f"\n💾 Memory Efficiency:")
print(f"   • Optimized: {unsloth_results['memory_gb']:.2f} GB")
print(f"   • Standard: {hf_results['memory_gb']:.2f} GB")
print(f"   • Difference: {abs(hf_results['memory_gb'] - unsloth_results['memory_gb']):.2f} GB ({abs(mem_saved):.1f}%)")
print(f"\n✅ Model Quality:")
print(f"   • Optimized Loss: {unsloth_results['loss']:.4f}")
print(f"   • Standard Loss: {hf_results['loss']:.4f}")
print(f"   • Difference: {abs(hf_results['loss'] - unsloth_results['loss']):.4f}")
if abs(hf_results['loss'] - unsloth_results['loss']) < 0.01:
    print(f"   • Both approaches produce equivalent quality results")

# Visualization
%matplotlib inline
sns.set_style("whitegrid")
fig, axes = plt.subplots(1, 3, figsize=(16, 5))
fig.suptitle('⚡ Optimization Impact: Standard vs Optimized Configuration', fontsize=16, fontweight='bold')

colors = ['#f093fb', '#4facfe']

# Plot 1: Training Time
ax1 = axes[0]
bars1 = ax1.barh(comparison_df['method'], comparison_df['time_seconds'], color=colors)
ax1.set_xlabel('Training Time (seconds)', fontweight='bold')
ax1.set_title('⏱️ Training Speed', fontweight='bold')
ax1.invert_yaxis()
for bar, time_val in zip(bars1, comparison_df['time_seconds']):
    ax1.text(bar.get_width(), bar.get_y() + bar.get_height()/2, f' {time_val:.1f}s', va='center', fontweight='bold')

# Plot 2: Memory Usage
ax2 = axes[1]
bars2 = ax2.barh(comparison_df['method'], comparison_df['memory_gb'], color=colors)
ax2.set_xlabel('Peak GPU Memory (GB)', fontweight='bold')
ax2.set_title('💾 Memory Usage', fontweight='bold')
ax2.invert_yaxis()
for bar, mem_val in zip(bars2, comparison_df['memory_gb']):
    ax2.text(bar.get_width(), bar.get_y() + bar.get_height()/2, f' {mem_val:.2f} GB', va='center', fontweight='bold')

# Plot 3: Speedup
ax3 = axes[2]
bars3 = ax3.bar(comparison_df['method'], comparison_df['speedup'], color=colors)
ax3.set_ylabel('Speedup (× faster)', fontweight='bold')
ax3.set_title('🚀 Speed Improvement', fontweight='bold')
ax3.axhline(y=1.0, color='red', linestyle='--', alpha=0.7, label='Baseline')
ax3.legend()
for bar, speedup_val in zip(bars3, comparison_df['speedup']):
    ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height(), f'{speedup_val:.2f}×', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.savefig('finetuning_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n💾 Chart saved: finetuning_comparison.png\n")
print("="*80 + "\n")


---

# 🎉 Summary

## What You Discovered

You just ran a **production-grade comparison** on YOUR GPU!

### ✅ Performance Characteristics:

| Metric | Optimized (Unsloth) | Standard (HF PEFT) | Difference |
|--------|---------------------|--------------------|-----------| 
| **Speed** | Faster | Standard | 2-3× with optimizations |
| **Memory** | Lower | Standard | 20-40% reduction |
| **Quality** | ✅ Equivalent | ✅ Equivalent | No trade-off |
| **Flexibility** | Supported models | Any model | HF more flexible |
| **Compatibility** | Limited architectures | Universal | HF more compatible |

**Key Insight:** Optimization layers can significantly improve performance while maintaining quality, but come with model support trade-offs.

---

## 🎯 When to Use Each Approach

### Choose Standard HuggingFace PEFT when:
- ✅ Working with new/custom model architectures
- ✅ Need maximum flexibility for research
- ✅ Require multi-GPU training with DeepSpeed/FSDP
- ✅ Using models not yet supported by optimization frameworks
- ✅ Need fine-grained control over training loops

### Choose Optimized Frameworks (Unsloth) when:
- ✅ Using supported models (Llama, Mistral, Qwen, etc.)
- ✅ Single-GPU training focused on speed
- ✅ Memory constraints are critical
- ✅ Cost optimization is important
- ✅ Production pipelines with consistent model choices

**Both approaches rely on HuggingFace's excellent transformers foundation.**

---

## 🚀 Next Steps

1. **Scale up** - Train 7B or 13B models
2. **Your data** - Use your own dataset  
3. **Production** - Export configs and deploy

### Learn More:
- **Unsloth**: https://github.com/unslothai/unsloth
- **Brev**: https://brev.dev
- **Discord**: https://discord.gg/NVDyv7TUgJ

---

**Questions? Feedback?** Join us on [Discord](https://discord.gg/NVDyv7TUgJ) or [X/Twitter](https://x.com/brevdev)

<div style="background: linear-gradient(135deg, #f093fb 0%, #f5576c 100%); padding: 30px; border-radius: 10px; color: white; text-align: center; margin-top: 30px;">
  <h2 style="color: white; margin-top: 0;">🎯 Ready for Production!</h2>
  <p style="font-size: 18px; margin-bottom: 0;">
    <strong>Go build something amazing. 🚀</strong>
  </p>
</div>

---

**Built with ❤️ by Brev**
